In [2]:
# Install all the required libraries
%pip install pandas
%pip install scikit-learn
%pip install request
%pip install matplotlib
%pip install numpy
%pip install joblib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

# Read the csv file
df = pd.read_csv("../datasets/weather.csv")

In [4]:
# Show quantity of each weather
print(df["Weather"].value_counts())

Weather
Mainly Clear                               2106
Mostly Cloudy                              2069
Cloudy                                     1728
Clear                                      1326
Snow                                        390
Rain                                        306
Rain Showers                                188
Fog                                         150
Rain,Fog                                    116
Drizzle,Fog                                  80
Snow Showers                                 60
Drizzle                                      41
Snow,Fog                                     37
Snow,Blowing Snow                            19
Rain,Snow                                    18
Thunderstorms,Rain Showers                   16
Haze                                         16
Drizzle,Snow,Fog                             15
Freezing Rain                                14
Freezing Drizzle,Snow                        11
Freezing Drizzle                

In [5]:
# show a resumen of dataframe
df.head()

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog


In [6]:
# delete the column 'Date/Time'
df = df.drop(columns=["Date/Time"])

In [7]:
# Rename the columns
df = df.rename(
    columns={
        "Temp_C": "Temp",
        "Dew Point Temp_C": "DewPoint",
        "Rel Hum_%": "Humidity",
        "Wind Speed_km/h": "WindSpeed",
        "Visibility_km": "Visibility",
        "Press_kPa": "Pressure",
    }
)

# show columns
df.columns

Index(['Temp', 'DewPoint', 'Humidity', 'WindSpeed', 'Visibility', 'Pressure',
       'Weather'],
      dtype='object')

In [8]:
from pprint import pprint

# Get the max and min of each column
ls = ["Temp", "DewPoint", "Humidity", "WindSpeed", "Visibility", "Pressure"]
data = {}

for i in ls:
    max = df[i].max()
    min = df[i].min()

    data[i] = {"max": max, "min": min}

pprint(data)

{'DewPoint': {'max': 24.4, 'min': -28.5},
 'Humidity': {'max': 100, 'min': 18},
 'Pressure': {'max': 103.65, 'min': 97.52},
 'Temp': {'max': 33.0, 'min': -23.3},
 'Visibility': {'max': 48.3, 'min': 0.2},
 'WindSpeed': {'max': 83, 'min': 0}}


In [9]:
# assign the values of the variables X and Y
X = df.drop("Weather", axis=1).values

Y = df["Weather"].values

In [10]:
# split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import mlflow
from mlflow.models.signature import infer_signature

# Create a model and train it with logistic regression
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("weather")

with mlflow.start_run(log_system_metrics=True):
    mlflow.log_input(mlflow.data.from_pandas(df), "weather.csv")

    # Create a pipeline
    pipeline = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler()),
            ("minmax", MinMaxScaler()),
            ("logreg", LogisticRegression()),
        ]
    )

    params = [
        # Solver newton-cg supports only dual=False, got dual=True
        {
            "logreg__penalty": ["l2"],
            "logreg__dual": [False],
            "logreg__n_jobs": [1, 10, 20, 25, 50, 100],
            "logreg__tol": [0.0001],
            "logreg__C": [1.0],
            "logreg__fit_intercept": [True, False],
            "logreg__solver": ["newton-cg"],
        },
        {
            "logreg__penalty": ["l2"],
            "logreg__dual": [False],
            "logreg__n_jobs": [1, 10, 20, 25, 50, 100],
            "logreg__tol": [0.0001],
            "logreg__C": [1.0],
            "logreg__fit_intercept": [True, False],
            "logreg__solver": ["newton-cholesky", "sag"],
        },
         {
            "logreg__penalty": ["l2"],
            "logreg__dual": [False],
            "logreg__n_jobs": [1, 10, 20, 25, 50, 100],
            "logreg__tol": [0.0001],
            "logreg__C": [1.0],
            "logreg__fit_intercept": [True, False],
            "logreg__solver": ["lbfgs"],
        },
        {
            "logreg__penalty": ["elasticnet"],
            "logreg__dual": [False],
            "logreg__n_jobs": [1, 10, 20, 25, 50, 100],
            "logreg__tol": [0.0001],
            "logreg__l1_ratio": [0.5],
            "logreg__C": [1.0],
            "logreg__fit_intercept": [True, False],
            "logreg__solver": ["saga"],
        },
    ]
    p = GridSearchCV(pipeline, param_grid=params, cv=5, n_jobs=-1, verbose=1)

    # Train the model
    p.fit(X_train, Y_train)

    # add the parameters to the model
    for key, value in p.best_params_.items():
        mlflow.log_param(key, value)

    # add the metrics to the model
    accuracy = accuracy_score(Y_test, p.predict(X_test))
    conf_matrix = confusion_matrix(Y_test, p.predict(X_test))
    f1 = f1_score(Y_test, p.predict(X_test), average="weighted")

    # add the metrics to the model
    mlflow.log_metric("accuracy_score", accuracy)
    # mlflow.log_metric("confusion_matrix", conf_matrix)
    mlflow.log_metric("f1_score", f1)

    # add signature to the model
    signature = infer_signature(X_train, p.predict(X_train))
    model_info = mlflow.sklearn.log_model(
        sk_model=p,
        artifact_path="weather_model",
        signature=signature,
        registered_model_name="weather_model",
    )
    mlflow.log_artifact("../streamlit/model/weather.pkl")

2024/10/22 12:07:43 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/10/22 12:07:43 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
c:\Users\diogenes monegro\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers 

Fitting 5 folds for each of 60 candidates, totalling 300 fits


Registered model 'weather_model' already exists. Creating a new version of this model...
2024/10/22 12:09:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: weather_model, version 2
Created version '2' of model 'weather_model'.
2024/10/22 12:09:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run nimble-koi-826 at: http://localhost:5000/#/experiments/428261603326214060/runs/334d52277a4e426c923b7f02091075c3.
2024/10/22 12:09:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/428261603326214060.
2024/10/22 12:09:09 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/10/22 12:09:09 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [17]:
# Make predictions
Y_pred = p.predict([X_test[0]])

# Show the prediction
print(Y_pred, Y_test[0])

['Cloudy'] Mostly Cloudy


In [18]:
# check accuracy of model
print("Accuracy Score", accuracy_score(Y_test, p.predict(X_test)))

print("Confusion Matrix: ", confusion_matrix(Y_test, p.predict(X_test)))

print("F1 Score:", f1_score(Y_test, p.predict(X_test), average="weighted"))

Accuracy Score 0.38531587933978373
Confusion Matrix:  [[ 92  34   0 ...   0   0   0]
 [ 50 143   0 ...   0   0   0]
 [  0   6   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   1   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
F1 Score: 0.3672052023327009


In [19]:
# export the model
import joblib

joblib.dump(p, "../streamlit/model/weather.pkl")

['../streamlit/model/weather.pkl']